In [1]:
import os
import sys

In [2]:
from langchain_community.document_loaders import UnstructuredFileLoader,UnstructuredPDFLoader,DirectoryLoader,PyPDFDirectoryLoader

In [3]:
import os
import unstructured
from langchain_community.document_loaders import DirectoryLoader
from pdf2image import convert_from_path 
import pytesseract

In [ ]:
class CustomPDFLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        pdf_images = convert_from_path(self.file_path)
        # with open(self.file_path) as doc:
        #     text = ""
        #     for page in doc:
        #         text += page.get_text()
        text = ""
        for image in pdf_images:
            text +=pytesseract.image_to_string(image)
        return {"page_content": text, "metadata": {"source": self.file_path}}

class CustomDirectoryLoader(DirectoryLoader):
    def __init__(self, directory, glob="*.pdf"):
        self.directory = directory
        self.glob = glob

    def load(self):
        documents = []
        for filename in os.listdir(self.directory):
            if filename.endswith(".pdf"):
                file_path = os.path.join(self.directory, filename)
                pdf_loader = CustomPDFLoader(file_path)
                documents.append(pdf_loader.load())
        return documents


path = os.path.join(os.path.dirname('./'),'alphabet')
loader = CustomDirectoryLoader(directory=path)
documents = loader.load()
print(documents)


[{'page_content': 'NAVEEN KUMAR\n\nSoftware Developer Engineer ¢ 9398490728 « sayhai2naveenkumar@gmail.com\n\nwww.linkedin.com/in/naveen0003 « https://github.com/SpideyO3\n\nSUMMARY\n\nSoftware Developer with nearly 5 years of relevant experience.\n\nProficient in Django and Flask Python frameworks and Object-Relational Mapping (ORM).\nHands-on experience with machine learning algorithms, including K-Nearest Neighbors (KNN),\nLinear Regression, and Neural Networks.\n\nExtensive expertise in designing REST API endpoints and server architecture.\n\nSolid working knowledge of AWS services, including $3, EC2, and Rekognition.\n\nExperienced in querying SQL databases.\n\nAdopts a Test-Driven Development (TDD) approach, with unit and integration testing skills.\nDemonstrated experience in building chatbots with custom integrations is an added advantage.\nStrong communication, collaboration, and interpersonal skills, with a proven ability to quickly\ngrasp and effectively utilize new technolo

In [18]:
type(documents[0])

dict

In [16]:
os.listdir(path)

['Naveen Kumar 2024.pdf',
 'Praveen_13Yrs_Datascience_AI.pdf',
 'Naveen Kumar 2024.pdf:Zone.Identifier',
 'alp_data.pdf']

In [4]:
import pickle
# pickle.dump(documents, open('documents.pkl', 'wb'))
documents = pickle.load(open('documents.pkl', 'rb'))

In [5]:
from langchain_core.documents.base import Document
documents_Doc = [Document(page_content=x['page_content'],metadata=x['metadata']) for x in documents]

In [11]:
documents_Doc

[Document(metadata={'source': './alphabet/Naveen Kumar 2024.pdf'}, page_content='NAVEEN KUMAR\n\nSoftware Developer Engineer ¢ 9398490728 « sayhai2naveenkumar@gmail.com\n\nwww.linkedin.com/in/naveen0003 « https://github.com/SpideyO3\n\nSUMMARY\n\nSoftware Developer with nearly 5 years of relevant experience.\n\nProficient in Django and Flask Python frameworks and Object-Relational Mapping (ORM).\nHands-on experience with machine learning algorithms, including K-Nearest Neighbors (KNN),\nLinear Regression, and Neural Networks.\n\nExtensive expertise in designing REST API endpoints and server architecture.\n\nSolid working knowledge of AWS services, including $3, EC2, and Rekognition.\n\nExperienced in querying SQL databases.\n\nAdopts a Test-Driven Development (TDD) approach, with unit and integration testing skills.\nDemonstrated experience in building chatbots with custom integrations is an added advantage.\nStrong communication, collaboration, and interpersonal skills, with a proven 

In [ ]:
## Converting content into dense vector embeddings 
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma 

#Split and chunk the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents_Doc)


# Add the chunks to vector database, which takes the model for creating the embeddings.
# vector_db = Chroma.from_documents(
#                                     documents=chunks, 
#                                     embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
#                                     persist_directory="resume_db_2"
#                                 )

vector_db = Chroma(persist_directory='resume_db_2', embedding_function=OllamaEmbeddings(model="nomic-embed-text", show_progress=True))
#https://github.com/hwchase17/chroma-langchain/blob/master/persistent-qa.ipynb

/tmp/ipykernel_39607/2258722662.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
OllamaEmbeddings: 100%|██████████| 49/49 [00:05<00:00,  9.17it/s]
/tmp/ipykernel_39607/2258722662.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory='resume_db_2', embedding_function=OllamaEmb

In [14]:
chunks

[Document(metadata={'source': './alphabet/Naveen Kumar 2024.pdf'}, page_content='NAVEEN KUMAR\n\nSoftware Developer Engineer ¢ 9398490728 « sayhai2naveenkumar@gmail.com\n\nwww.linkedin.com/in/naveen0003 « https://github.com/SpideyO3\n\nSUMMARY\n\nSoftware Developer with nearly 5 years of relevant experience.\n\nProficient in Django and Flask Python frameworks and Object-Relational Mapping (ORM).\nHands-on experience with machine learning algorithms, including K-Nearest Neighbors (KNN),\nLinear Regression, and Neural Networks.\n\nExtensive expertise in designing REST API endpoints and server architecture.\n\nSolid working knowledge of AWS services, including $3, EC2, and Rekognition.\n\nExperienced in querying SQL databases.\n\nAdopts a Test-Driven Development (TDD) approach, with unit and integration testing skills.\nDemonstrated experience in building chatbots with custom integrations is an added advantage.\nStrong communication, collaboration, and interpersonal skills, with a proven 

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate

local_llm = 'llama3.2'
llm_model = ChatOllama(model=local_llm)

QUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template='You are an AI assistant. You will be given a question. You must generate a detailed and long answer. You can only ask one question per turn. While answering think step-by-step and justify your answer.'
)

# RAG Prompt
template = """Answer the question based ONLY on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm=llm_model,prompt=QUERY_PROMPT)

/tmp/ipykernel_39607/4069781448.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm_model = ChatOllama(model=local_llm)


In [12]:
user_question = "Give me the list of companies Praveen Kumar worked for"
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm_model 
    | StrOutputParser()
)

response = chain.invoke(user_question)

print(response)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 43.64it/s]


I don't have access to that information. The text you provided appears to be an excerpt from Alphabet Inc.'s (Google's) 2022 annual report, but I couldn't find any mention of a person named Praveen Kumar working for the company. If you could provide more context or clarify who Praveen Kumar is, I may be able to help you better.


In [10]:
user_question = "do you have any information about Naveen"
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm_model 
    | StrOutputParser()
)

response = chain.invoke(user_question)

print(response)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 61.04it/s]


A LinkedIn profile!

Yes, I can extract some information from the profile. Here's a summary:

**Name:** Naveen
**Location:** Kurnool, Andhra Pradesh, India
**Age:** Not specified (based on the education history)
**Education:**

* IIIT RGUKT, Computer Science and Engineering (CGPA 8.5)
* Pre University Course (CGPA 8.5)
* SSC (CGPA 9.0)

**Work Experience:**

None mentioned in this profile.

**Projects:**

1. **Reddit Bot with NLP**: Created a chatbot to automate social media interactions on the Reddit platform using PRAW.
2. **Task Flow**: Designed a task flow system that can accommodate customized series of steps, used Elastic search NoSQL database, and followed agile methodology.
3. **CIS & MMB**: Streamlined the process of building and validating changes, ensuring everything is ready for final shipment to end customer.
4. **Handwritten Digit Recognizer using CNN**: Developed a model using MNIST dataset aimed at identifying handwritten digits (0-9) using Convolution Neural Networks.


In [13]:
user_question = "Tell me something about alphabet company"
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm_model 
    | StrOutputParser()
)

response = chain.invoke(user_question)

print(response)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 20.18it/s]


Alphabet Inc. is a multinational conglomerate holding company headquartered in Mountain View, California. It was founded on September 15, 2015, by Larry Page and Sergey Brin, who previously co-founded Google.

Here are some key facts about Alphabet Company:

**Business Segments:**

1. **Google**: The largest segment of Alphabet, responsible for online advertising, cloud computing, hardware, and software.
2. **YouTube**: A video-sharing platform acquired by Google in 2006.
3. **Verizon**: A telecommunications company owned by Alphabet, which provides wireless communication services.
4. **Calico**: A life sciences company focused on aging research and longevity.
5. **X Development** (formerly Google X): An advanced technology research division working on cutting-edge projects like self-driving cars and lunar exploration.
6. **Waymo** (formerly Google Self-Driving Car Project): A autonomous driving project that has developed fully self-driving cars for commercial use.

**Parent Companies:

In [20]:
user_question = "tell me about Naveen technical competency"
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm_model 
    | StrOutputParser()
)

response = chain.invoke(user_question)

print(response)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 67.50it/s]


There is no information provided about a person named Naveen or their technical competency. The text you provided earlier appears to be an excerpt from Alphabet Inc.'s annual report, and it does not mention anyone named Naveen.

If you could provide more context or clarify who Naveen is and what their role or expertise might be, I'd be happy to try and help you with your question.
